In [ ]:
import numpy as np
import open3d as o3d
import pymeshfix as mf

data = np.load('3d_shape_points_data.npz')
points = data['points']

# Create an Open3D point cloud object
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points)
downpcd = pcd.voxel_down_sample(voxel_size=0.05)# reduces the number of points in the point cloud by grouping nearby points into voxels (small 3D grid cells of size 0.05).
pcd=downpcd
o3d.visualization.draw_geometries([pcd])

In [24]:
# remove outliers if any
# cl, ind = pcd.remove_radius_outlier(nb_points=2, radius=0.1)
cl, ind = pcd.remove_statistical_outlier(nb_neighbors=20, std_ratio=1)
filtered_pcd = pcd.select_by_index(ind)
o3d.visualization.draw_geometries([filtered_pcd], window_name="Filtered Point Cloud")
pcd=filtered_pcd

In [25]:
# creating a mesh from the point cloud using alpha shape
alpha = 0.5
print(f"alpha={alpha:.3f}")
mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_alpha_shape(pcd, alpha)
mesh.compute_vertex_normals()
o3d.visualization.draw_geometries([mesh])

alpha=0.500


In [ ]:
mesh = mesh.filter_smooth_laplacian(number_of_iterations=100)

mesh.compute_vertex_normals()
o3d.visualization.draw_geometries([mesh])

point_cloud = mesh.sample_points_uniformly(number_of_points=10000)
o3d.visualization.draw_geometries([point_cloud])

In [34]:
# creating a mesh from the point cloud using poisson
pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid( radius=0.1, max_nn=20))
mesh , densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd, depth=9)
o3d.visualization.draw_geometries([mesh])


In [37]:
constraint_ids = np.random.choice(len(mesh.vertices), size=10, replace=False)  # Select 10 random points

# Convert indices to Open3D IntVector format
constraint_indices_o3d = o3d.utility.IntVector(constraint_ids)

# Get their original positions
constraint_positions = np.asarray(mesh.vertices)[constraint_ids]  # Extract original positions
constraint_positions_o3d = o3d.utility.Vector3dVector(constraint_positions)  # Convert to Open3D format

# Apply ARAP smoothing
smoothed_mesh = mesh.deform_as_rigid_as_possible(
    constraint_vertex_indices=constraint_indices_o3d,
    constraint_vertex_positions=constraint_positions_o3d,
    max_iter=50
)

# Visualize smoothed mesh
o3d.visualization.draw_geometries([smoothed_mesh])

RuntimeError: [Open3D Error] (class std::shared_ptr<class open3d::geometry::TriangleMesh> __cdecl open3d::geometry::TriangleMesh::DeformAsRigidAsPossible(const class std::vector<int,class std::allocator<int> > &,const class std::vector<class Eigen::Matrix<double,3,1,0,3,1>,class std::allocator<class Eigen::Matrix<double,3,1,0,3,1> > > &,unsigned __int64,enum open3d::geometry::MeshBase::DeformAsRigidAsPossibleEnergy,double) const) D:\a\Open3D\Open3D\cpp\open3d\geometry\TriangleMeshDeformation.cpp:82: Failed to build solver (factorize)


In [35]:
#visulising mesh created from poisson
mesh = mesh.filter_smooth_laplacian(number_of_iterations=100)

mesh.compute_vertex_normals()
# o3d.visualization.draw_geometries([mesh])

point_cloud = mesh.sample_points_uniformly(number_of_points=10000)
# o3d.visualization.draw_geometries([point_cloud])
cl, ind = point_cloud.remove_statistical_outlier(nb_neighbors=20, std_ratio=2.0)
pcd_clean = pcd.select_by_index(ind)
o3d.visualization.draw_geometries([pcd_clean])

In [30]:
# processing the mesh from the poisson
mesh.compute_vertex_normals()

# Compute curvature (using normal variation)
curvatures = np.linalg.norm(np.asarray(mesh.vertex_normals), axis=1)

# Set a threshold to remove high-curvature vertices
threshold = np.percentile(curvatures, 95)  # Remove top 5% of high-curvature vertices
outlier_indices = np.where(curvatures > threshold)[0]

# Keep only the remaining vertices
mesh_clean = mesh.remove_vertices_by_index(outlier_indices)

# Visualize cleaned mesh
o3d.visualization.draw_geometries([mesh_clean])

[Open3D WARNING] [AddGeometry] Invalid pointer. Possibly a null pointer or None was passed in.
[Open3D WARNING] [DrawGeometries] Failed adding geometry.
[Open3D WARNING] [DrawGeometries] Possibly due to bad geometry or wrong geometry type.


In [4]:
# radii = [0.5,1,2]  # Adjust radii carefully
radii = [0.1,0.2,0.3] 
mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(
    pcd, o3d.utility.DoubleVector(radii)
)

# Compute vertex normals
mesh.compute_vertex_normals()

o3d.visualization.draw_geometries([mesh])

In [14]:
mesh = mesh.filter_smooth_laplacian(number_of_iterations=100)

# Recompute normals for better visualization
mesh.compute_vertex_normals()
# mesh = mesh.fill_holes()
# mesh = mesh.filter_smooth_simple(number_of_iterations=5)
# mesh = mesh.filter_smooth_taubin(number_of_iterations=5)
# Visualize the smoothed mesh
o3d.visualization.draw_geometries([mesh])

In [17]:
point_cloud = mesh.sample_points_uniformly(number_of_points=10000)
o3d.visualization.draw_geometries([point_cloud])

In [18]:
import pymeshlab
remeshed_mesh = pymeshlab.resolve_self_intersection(mesh)
o3d.visualization.draw_geometries([remeshed_mesh])

AttributeError: module 'pymeshlab' has no attribute 'resolve_self_intersection'